In [28]:
import sys
sys.executable

'c:\\users\\jonathan\\python_env\\nba_stat\\scripts\\python.exe'

In [29]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler

import graphviz 
import itertools

In [30]:
def remove_slash_name(_df):
    df = _df.copy()
    df['Player'] = df['Player'].apply(lambda row: row.split('\\')[0])
    df['Player'] = df['Player'].apply(lambda row: row.split('*')[0])
    
    return df

def remove_rk(_df):
    df = _df.copy()
    return df.drop(['Rk'], axis=1)

def remove_team(_df):
    df = _df.copy()
    return df.drop(['Tm'], axis=1)

def remove_age(_df):
    df = _df.copy()
    return df.drop(['Age'], axis=1)

def remove_game(_df):
    df = _df.copy()
    return df.drop(['G'], axis=1)   

def remove_game_started(_df):
    df = _df.copy()
    return df.drop(['GS'], axis=1)   

def remove_min(_df):
    df = _df.copy()
    return df.drop(['MP'], axis=1)    

def extract_name_position(_df):
    df = _df.copy()
    return _df.drop(['Pos','Player'], axis=1), _df[['Player', 'Pos']]
    

def remove_nan(_df):
    df = _df.copy()
    return df.dropna(axis=1, how='all').fillna(0)

In [33]:
# we use pandas to load data directly from csv
df = pd.read_csv('adv_stats_95-96.csv')

# apply a couple of preprocessing function
df = remove_rk(remove_slash_name(df))
df = remove_age(remove_team(df))
df = remove_game(remove_min(df))

pos_used = ['PG','SG','SF','PF','C']
df = df.loc[df['Pos'].isin(pos_used)]
df = remove_nan(df)

df_old, _ = extract_name_position(df)

df = pd.read_csv('adv_stats_18-19.csv')

df = remove_rk(remove_slash_name(df))
df = remove_age(remove_team(df))
df = remove_game(remove_min(df))

pos_used = ['PG','SG','SF','PF','C']
df = df.loc[df['Pos'].isin(pos_used)]
df = remove_nan(df)

df_current, _ = extract_name_position(df)

scaler = MinMaxScaler()

performance_metric = {}
for metric in ['braycurtis', 'canberra', 'chebyshev', 'correlation', 'hamming', 'kulsinski', 'minkowski', 'rogerstanimoto', 'russellrao', 'sokalmichener', 'sokalsneath',
               'sqeuclidean']:
    neigh = NearestNeighbors(1, metric=metric)
    neigh.fit(df_current.values)

    dst, ids = neigh.kneighbors(df_old.values)
    dst = scaler.fit_transform(dst)
    performance_metric[metric] = np.mean(dst)

    
min_idx = min(performance_metric, key=performance_metric.get)
min_idx

'sqeuclidean'

In [11]:
def get_alike_player(name):
    # we use pandas to load data directly from csv
    df = pd.read_csv('adv_stats_95-96.csv')

    # apply a couple of preprocessing function
    df = remove_rk(remove_slash_name(df))
    df = remove_age(remove_team(df))
    df = remove_game(remove_min(df))

    pos_used = ['PG','SG','SF','PF','C']
    df = df.loc[df['Pos'].isin(pos_used)]
    df = remove_nan(df)

    df_jordan, _ = extract_name_position(df.loc[df['Player'] == name])
    df_jordan

    df = pd.read_csv('adv_stats_18-19.csv')

    df = remove_rk(remove_slash_name(df))
    df = remove_age(remove_team(df))
    df = remove_game(remove_min(df))

    pos_used = ['PG','SG','SF','PF','C']
    df = df.loc[df['Pos'].isin(pos_used)]
    df = remove_nan(df)

    df_current_player, df_current_names = extract_name_position(df)
    neigh = NearestNeighbors(5, metric=min_idx)
    neigh.fit(df_current_player.values)

    dst, ids = neigh.kneighbors(df_jordan.values)
    return df_current_names.iloc[ids[0]]

In [12]:
player_names = ['Michael Jordan', 'Charles Barkley', 'Shawn Bradley',
                'Muggsy Bogues', 'Patrick Ewing', 'Larry Johnson']
for p in player_names:
    print('Equivalent current player to {}'.format(p))
    print(get_alike_player(p))
    print()

Equivalent current player to Michael Jordan
             Player Pos
240     Paul George  SF
159   Stephen Curry  PG
191    Kevin Durant  PF
397   Kawhi Leonard  SF
401  Damian Lillard  PG

Equivalent current player to Charles Barkley
                 Player Pos
647  Karl-Anthony Towns   C
509        Jusuf Nurkic   C
574    Domantas Sabonis   C
661      Nikola Vucevic   C
161       Anthony Davis   C

Equivalent current player to Shawn Bradley
             Player Pos
36    Mohamed Bamba   C
258  JaMychal Green  PF
116  Wendell Carter   C
706     Ivica Zubac   C
448    JaVale McGee   C

Equivalent current player to Muggsy Bogues
                  Player Pos
170  Matthew Dellavedova  PG
634       Milos Teodosic  PG
225          Tim Frazier  PG
172  Matthew Dellavedova  PG
224          Tim Frazier  PG

Equivalent current player to Patrick Ewing
                Player Pos
655  Jonas Valanciunas   C
315        Serge Ibaka   C
554      Julius Randle  PF
656  Jonas Valanciunas   C
692     Chris

In [13]:
def get_alike_team(team, old='95-96', new='18-19'):
    # we use pandas to load data directly from csv
    df = pd.read_csv('adv_stats_{}.csv'.format(old))

    # apply a couple of preprocessing function
    df = remove_rk(remove_slash_name(df))
    df = df.loc[df['Tm'] == team]
    df = remove_age(remove_team(df))
    df = remove_game(remove_min(df))

    pos_used = ['PG','SG','SF','PF','C']
    df = df.loc[df['Pos'].isin(pos_used)]
    df = remove_nan(df)

    df_old_team, df_old_names = extract_name_position(df)

    df = pd.read_csv('adv_stats_{}.csv'.format(new))

    df = remove_rk(remove_slash_name(df))
    df = remove_age(remove_team(df))
    df = remove_game(remove_min(df))

    pos_used = ['PG','SG','SF','PF','C']
    df = df.loc[df['Pos'].isin(pos_used)]
    df = remove_nan(df)

    df_current_player, df_current_names = extract_name_position(df)
    neigh = NearestNeighbors(1, metric=min_idx)
    neigh.fit(df_current_player.values)

    dst, ids = neigh.kneighbors(df_old_team.values)
    result_df = pd.DataFrame({'{} players'.format(old) : (df_old_names.values)[:,0],
    '{} players'.format(new) : df_current_names.iloc[np.reshape(ids,-1)].values[:,0]})
    
    #return (df_old_names.values)[:,0], df_current_names.iloc[np.reshape(ids,-1)].values[:,0]
    return result_df

In [67]:
new = '18-19'
old = '95-96'
result_df = get_alike_team('CHI', new=new, old=old)
print('old player and current equivalent :')
result_df

old player and current equivalent :


,95-96 players,18-19 players
0,Randy Brown,George Hill
1,Jud Buechler,Corey Brewer
2,Jason Caffey,Guerschon Yabusele
3,James Edwards,Dzanan Musa
4,Jack Haley,Donatas Motiejunas
5,Ron Harper,Delon Wright
6,Michael Jordan,Paul George
7,Steve Kerr,Landry Shamet
8,Toni Kukoc,Jimmy Butler
9,Luc Longley,Jordan Bell


In [68]:
df = pd.read_csv('adv_stats_95-96.csv')

In [70]:
result_df.to_dict

<bound method DataFrame.to_dict of       95-96 players       18-19 players
0       Randy Brown         George Hill
1      Jud Buechler        Corey Brewer
2      Jason Caffey  Guerschon Yabusele
3     James Edwards         Dzanan Musa
4        Jack Haley  Donatas Motiejunas
5        Ron Harper        Delon Wright
6    Michael Jordan         Paul George
7        Steve Kerr       Landry Shamet
8        Toni Kukoc        Jimmy Butler
9       Luc Longley         Jordan Bell
10   Scottie Pippen        Eric Bledsoe
11    Dennis Rodman      DeAndre Jordan
12      John Salley  Isaiah Hartenstein
13  Dickey Simpkins  Isaiah Hartenstein
14  Bill Wennington          Sam Dekker>